In [15]:
import pandas as pd
import numpy as np
from pandas.plotting import table
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import scipy.stats as stats
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime

In [17]:
df_geral = pd.read_csv("clear-data/dados_tratados.csv")

In [18]:
df_geral

,ride_name,data_hora,wait_time,date,time,day_of_the_week,sale,holiday,holiday_season,season,...,HotWheels 1730,Acqua 1400,Acqua 1500,Madagascar 1500,Madagascar 1600,Madagascar 1700,Sonho 1800,Excalibur 1200,Excalibur 1430,Excalibur 1700
0,Autopista (bate-bate),2023-01-01 11:25:03,30,2023-01-01,11:25:03,sunday,NaN,new_year,summer vacation,high_season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Autopista (bate-bate),2023-01-01 11:30:04,30,2023-01-01,11:30:04,sunday,NaN,new_year,summer vacation,high_season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Autopista (bate-bate),2023-01-01 11:35:03,30,2023-01-01,11:35:03,sunday,NaN,new_year,summer vacation,high_season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Autopista (bate-bate),2023-01-01 11:40:03,30,2023-01-01,11:40:03,sunday,NaN,new_year,summer vacation,high_season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Autopista (bate-bate),2023-01-01 11:45:04,30,2023-01-01,11:45:04,sunday,NaN,new_year,summer vacation,high_season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663591,Xicaras Malucas,2023-12-31 18:35:06,10,2023-12-31,18:35:06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
663592,Xicaras Malucas,2023-12-31 18:40:06,10,2023-12-31,18:40:06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
663593,Xicaras Malucas,2023-12-31 18:45:06,10,2023-12-31,18:45:06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
663594,Xicaras Malucas,2023-12-31 18:50:06,10,2023-12-31,18:50:06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_geral['data_hora'] = pd.to_datetime(df_geral['data_hora'])
df_geral['date'] = df_geral['data_hora'].dt.date

In [20]:
tabela_horarios = pd.read_excel("bcw-operation-calendar.xlsx")
tabela_horarios["date"] = pd.to_datetime(tabela_horarios["date"])

In [35]:
# Cálculo dos percentis
p5 = 0
p90 = 0
p95 = 0

# Função de mapeamento de cores
def custom_color_map(value):
    if value > p95:
        return 'black'
    else:
        norm_value = (value - p5) / (p95 - p5)
        return plt.cm.RdYlGn_r(norm_value)

def gera_calendario(espera, tipo, metrica, mes, ano):
    if mes:
        espera = espera[(espera['month'] == mes) & (espera['year'] == ano)]
    else:
        espera = espera[espera['year'] == ano]
    
    # Reorganize o dataframe para a forma desejada
    pivot_df = espera.pivot_table(values='wait_time', index='week_of_year', columns='day_of_week', aggfunc=np.mean)

    # Reordene as colunas para começar na segunda-feira
    ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    # Crie um DataFrame temporário com todas as colunas ordenadas
    temp_df = pd.DataFrame(columns=ordered_days)
    
    for day in ordered_days:
        if day in pivot_df.columns:
            temp_df[day] = pivot_df[day]
        else:
            temp_df[day] = np.nan

    pivot_df = temp_df

    # Passo 1: Crie uma matriz de anotações
    annotation_array = []
    for week in pivot_df.index:
        annotation_row = []
        for day in ordered_days:
            date_row = espera[(espera['week_of_year'] == week) & (espera['day_of_week'] == day)]
            if not date_row.empty:
                day_of_month = date_row.iloc[0]['day']
                month = date_row.iloc[0]['month']
                wait_time = date_row.iloc[0]['wait_time']
                annotation_row.append(f"{day_of_month}/{month}\n{wait_time}")
            else:
                annotation_row.append('')
        annotation_array.append(annotation_row)

    annotation_array = np.array(annotation_array)
    
    # Inverter o DataFrame e a matriz de anotações
    pivot_df_inverted = pivot_df.iloc[::-1]
    annotation_array_inverted = annotation_array[::-1]

    altura = 0

    if tipo =='Mensal':
        altura = 10
    else:
        altura = 30
    
    plt.figure(figsize=(12, altura))

    # Anotação e cores
    for i, week in enumerate(pivot_df_inverted.index):
        for j, day in enumerate(pivot_df_inverted.columns):
            value = pivot_df_inverted.loc[week, day]
            if not np.isnan(value):
                color = 'white' if value > p90 or value < 17 else 'black'
                
                # A data acima do tempo médio de fila (fonte menor e sem negrito)
                day_of_month = annotation_array_inverted[i, j].split('\n')[0]
                plt.text(j + 0.8, i + 0.8, f"{day_of_month}", ha='center', va='center', fontsize=10, fontweight='normal', color=color)

                # O tempo médio de fila (fonte maior e negrito)
                wait_time = annotation_array_inverted[i, j].split('\n')[1]
                wait_time = int(float(wait_time))  # Converter para float e, em seguida, para inteiro
            
                plt.text(j + 0.5, i + 0.4, f"{wait_time}", ha='center', va='center', fontsize=18, fontweight='bold', color=color)

                plt.fill_betweenx([i, i+1], j, j+1, color=custom_color_map(value))
                plt.plot([j, j+1, j+1, j, j], [i, i, i+1, i+1, i], color='white', linewidth=1.5)

    plt.xticks(ticks=np.arange(0.5, len(pivot_df_inverted.columns)), labels=pivot_df_inverted.columns, rotation=45)
    plt.yticks(ticks=np.arange(0.5, len(pivot_df_inverted.index)), labels=pivot_df_inverted.index)
    titulo = ""
    
    if metrica == 'mean':
        titulo = "Calendário de Tempo de Espera Médio"
    elif metrica == 'max':
        titulo = "Calendário de Tempo de Espera Máximo"
    elif metrica == 'median':
        titulo = "Calendário de Tempo de Espera Mediano"
    elif metrica == 'mode':
        titulo = "Calendário de Tempo de Espera Modal"
    elif metrica == 'std':
        titulo = "Calendário dos Desvios Padrão do Tempo de Espera"
    
    plt.title(titulo)
    plt.xlabel("Dia da Semana")
    plt.ylabel("Semana do Ano")

    now = datetime.now()
    data_hora_formatada = now.strftime('%Y-%m-%d-%H%M')

    nome_arquivo = ''
    
    if tipo == 'Mensal':
        nome_arquivo=f'{metrica}_daily_wait_time_{mes}{ano}-{data_hora_formatada}.png'
    else:
        nome_arquivo=f'{metrica}_daily_wait_time_{ano}-{data_hora_formatada}.png'

    plt.savefig(nome_arquivo)

    plt.show()

In [ ]:
def gera_calendario_medias_mensal(espera, p5, p90, p95): 
    # Reorganize o dataframe para a forma desejada
    pivot_df = espera.pivot_table(values='wait_time', index='week_of_year', columns='day_of_week', aggfunc=np.mean)

    # Reordene as colunas para começar na segunda-feira
    ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    # Crie um DataFrame temporário com todas as colunas ordenadas
    temp_df = pd.DataFrame(columns=ordered_days)
    
    for day in ordered_days:
        if day in pivot_df.columns:
            temp_df[day] = pivot_df[day]
        else:
            temp_df[day] = np.nan

    pivot_df = temp_df

    # Passo 1: Crie uma matriz de anotações
    annotation_array = []
    for week in pivot_df.index:
        annotation_row = []
        for day in ordered_days:
            date_row = espera[(espera['week_of_year'] == week) & (espera['day_of_week'] == day)]
            if not date_row.empty:
                day_of_month = date_row.iloc[0]['day']
                month = date_row.iloc[0]['month']
                wait_time = date_row.iloc[0]['wait_time']
                annotation_row.append(f"{day_of_month}/{month}\n{wait_time}")
            else:
                annotation_row.append('')
        annotation_array.append(annotation_row)

    annotation_array = np.array(annotation_array)
    
    # Inverter o DataFrame e a matriz de anotações
    pivot_df_inverted = pivot_df.iloc[::-1]
    annotation_array_inverted = annotation_array[::-1]

    altura = 0

    if tipo =='Mensal':
        altura = 10
    else:
        altura = 30
    
    plt.figure(figsize=(12, altura))

    # Anotação e cores
    for i, week in enumerate(pivot_df_inverted.index):
        for j, day in enumerate(pivot_df_inverted.columns):
            value = pivot_df_inverted.loc[week, day]
            if not np.isnan(value):
                color = 'white' if value > p90 or value < 17 else 'black'
                
                # A data acima do tempo médio de fila (fonte menor e sem negrito)
                day_of_month = annotation_array_inverted[i, j].split('\n')[0]
                plt.text(j + 0.8, i + 0.8, f"{day_of_month}", ha='center', va='center', fontsize=10, fontweight='normal', color=color)

                # O tempo médio de fila (fonte maior e negrito)
                wait_time = annotation_array_inverted[i, j].split('\n')[1]
                wait_time = int(float(wait_time))  # Converter para float e, em seguida, para inteiro
            
                plt.text(j + 0.5, i + 0.4, f"{wait_time}", ha='center', va='center', fontsize=18, fontweight='bold', color=color)

                plt.fill_betweenx([i, i+1], j, j+1, color=custom_color_map(value))
                plt.plot([j, j+1, j+1, j, j], [i, i, i+1, i+1, i], color='white', linewidth=1.5)

    plt.xticks(ticks=np.arange(0.5, len(pivot_df_inverted.columns)), labels=pivot_df_inverted.columns, rotation=45)
    plt.yticks(ticks=np.arange(0.5, len(pivot_df_inverted.index)), labels=pivot_df_inverted.index)
    titulo = ""
    
    if metrica == 'mean':
        titulo = "Calendário de Tempo de Espera Médio"
    elif metrica == 'max':
        titulo = "Calendário de Tempo de Espera Máximo"
    elif metrica == 'median':
        titulo = "Calendário de Tempo de Espera Mediano"
    elif metrica == 'mode':
        titulo = "Calendário de Tempo de Espera Modal"
    elif metrica == 'std':
        titulo = "Calendário dos Desvios Padrão do Tempo de Espera"
    
    plt.title(titulo)
    plt.xlabel("Dia da Semana")
    plt.ylabel("Semana do Ano")

    now = datetime.now()
    data_hora_formatada = now.strftime('%Y-%m-%d-%H%M')

    nome_arquivo = ''
    
    if tipo == 'Mensal':
        nome_arquivo=f'{metrica}_daily_wait_time_{mes}{ano}-{data_hora_formatada}.png'
    else:
        nome_arquivo=f'{metrica}_daily_wait_time_{ano}-{data_hora_formatada}.png'

    plt.savefig(nome_arquivo)

    plt.show()

In [22]:
lista_atracoes = [
    'Autopista (bate-bate)', 'Baby Elefante', 'Barco Pirata',
    'Betinho Carrero 4D', 'Big Tower', 'Carrossel Veneziano',
    'Madagascar Crazy River Adventure!', 'Ferrovia DinoMagic',
    'FireWhip', 'Montanha-russa Dum Dum', 'Pedalinho', 'Raskapuska',
    'Rebulico', 'Roda-Gigante', 'Spin Blast', 'Star Mountain',
    'Super Soaker Splash', 'Tchibum', 'Tigor Mountain',
    'Trenzinho Vila Esperanca', 'Xicaras Malucas'
]

for i, atracao in enumerate(lista_atracoes, 1):
    print(f"{i}: {atracao}")
    
selecao_usuario = input("Digite os números das atrações que deseja selecionar, separados por vírgula (ex.: 1, 2, 3). Digite 0 para selecionar todas: ")

# Processar a entrada do usuário
if selecao_usuario == "0":
    df_filtrado = df_geral[df_geral['ride_name'].isin(lista_atracoes)]
else:
    indices_selecionados = [int(x) - 1 for x in selecao_usuario.split(',') if x.isdigit()]
    atracoes_selecionadas = [lista_atracoes[i] for i in indices_selecionados if i < len(lista_atracoes)]
    df_filtrado = df_geral[df_geral['ride_name'].isin(atracoes_selecionadas)]

# Mostrar o DataFrame filtrado
print(df_filtrado)

1: Autopista (bate-bate)
2: Baby Elefante
3: Barco Pirata
4: Betinho Carrero 4D
5: Big Tower
6: Carrossel Veneziano
7: Madagascar Crazy River Adventure!
8: Ferrovia DinoMagic
9: FireWhip
10: Montanha-russa Dum Dum
11: Pedalinho
12: Raskapuska
13: Rebulico
14: Roda-Gigante
15: Spin Blast
16: Star Mountain
17: Super Soaker Splash
18: Tchibum
19: Tigor Mountain
20: Trenzinho Vila Esperanca
21: Xicaras Malucas
Digite os números das atrações que deseja selecionar, separados por vírgula (ex.: 1, 2, 3). Digite 0 para selecionar todas: 0
                    ride_name           data_hora  wait_time        date  \
0       Autopista (bate-bate) 2023-01-01 11:25:03         30  2023-01-01   
1       Autopista (bate-bate) 2023-01-01 11:30:04         30  2023-01-01   
2       Autopista (bate-bate) 2023-01-01 11:35:03         30  2023-01-01   
3       Autopista (bate-bate) 2023-01-01 11:40:03         30  2023-01-01   
4       Autopista (bate-bate) 2023-01-01 11:45:04         30  2023-01-01   
...     

In [23]:
espera_media = df_filtrado.groupby('date')['wait_time'].mean().reset_index()

In [24]:
espera_media

,date,wait_time
0,2023-01-01,45.302395
1,2023-01-02,18.818849
2,2023-01-03,31.461587
3,2023-01-04,28.063549
4,2023-01-05,67.809495
...,...,...
380,2024-03-27,26.372657
381,2024-03-28,31.058946
382,2024-03-29,49.549814
383,2024-03-30,54.840999


In [25]:
espera_media['wait_time'] = espera_media['wait_time'].round(0)

In [26]:
espera_media

,date,wait_time
0,2023-01-01,45.0
1,2023-01-02,19.0
2,2023-01-03,31.0
3,2023-01-04,28.0
4,2023-01-05,68.0
...,...,...
380,2024-03-27,26.0
381,2024-03-28,31.0
382,2024-03-29,50.0
383,2024-03-30,55.0


In [28]:
espera_media['date'] = pd.to_datetime(espera_media['date'])
espera_media['year'] = espera_media['date'].dt.year
espera_media['month'] = espera_media['date'].dt.month
espera_media['day'] = espera_media['date'].dt.day

In [29]:
espera_media = pd.merge(espera_media, tabela_horarios, on="date", how="left")

# Adicione colunas para o dia da semana e a semana do ano
espera_media['day_of_week'] = espera_media['date'].dt.day_name()
espera_media['week_of_year'] = espera_media['date'].dt.isocalendar().week

# Identifique os registros que atendem ao critério
primeiros_dias_janeiro = (espera_media['month'] == 1) & (espera_media['week_of_year'] > 50)

# Atribua o valor 0 à coluna 'week_of_year' para esses registros
espera_media.loc[primeiros_dias_janeiro, 'week_of_year'] = 0

espera_media

,date,wait_time,year,month,day,day_of_the_week,park_open,park_opening_time,park_closing_time,sale,...,Madagascar 1600,Madagascar 1700,Sonho 1800,Excalibur 1200,Excalibur 1430,Excalibur 1700,Shrek 1900,Trolls 1900,day_of_week,week_of_year
0,2023-01-01,45.0,2023,1,1,sunday,True,10:00:00,19:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday,0
1,2023-01-02,19.0,2023,1,2,monday,True,10:00:00,19:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday,1
2,2023-01-03,31.0,2023,1,3,tuesday,True,10:00:00,19:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tuesday,1
3,2023-01-04,28.0,2023,1,4,wednesday,True,10:00:00,19:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,1
4,2023-01-05,68.0,2023,1,5,thursday,True,10:00:00,19:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,2024-03-27,26.0,2024,3,27,wednesday,True,10:00:00,19:00:00,NaN,...,True,False,True,True,False,False,False,True,Wednesday,13
381,2024-03-28,31.0,2024,3,28,thursday,True,10:00:00,19:00:00,NaN,...,True,False,True,True,False,False,False,True,Thursday,13
382,2024-03-29,50.0,2024,3,29,friday,True,10:00:00,19:00:00,NaN,...,True,False,True,True,False,False,False,True,Friday,13
383,2024-03-30,55.0,2024,3,30,saturday,True,10:00:00,19:00:00,NaN,...,True,False,True,True,False,False,False,True,Saturday,13


In [30]:
# Reorganize o dataframe para a forma desejada
pivot_df = espera_media.pivot_table(values='wait_time', index='week_of_year', columns='day_of_week', aggfunc=np.mean)

# Reordene as colunas para começar na segunda-feira
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
pivot_df = pivot_df[ordered_days]

In [42]:
# Cálculo dos percentis
p5 = np.percentile(espera_media['wait_time'], 5)
p90 = np.percentile(espera_media['wait_time'], 90)
p95 = np.percentile(espera_media['wait_time'], 95)

print(p5, p90, p95)

14.0 57.0 62.799999999999955


In [40]:
espera_media_2023 = espera_media[(espera_media['year'] == 2023)]
espera_media_2024 = espera_media[(espera_media['year'] == 2024)]

In [41]:
espera_media_2024

,date,wait_time,year,month,day,day_of_the_week,park_open,park_opening_time,park_closing_time,sale,...,Madagascar 1600,Madagascar 1700,Sonho 1800,Excalibur 1200,Excalibur 1430,Excalibur 1700,Shrek 1900,Trolls 1900,day_of_week,week_of_year
294,2024-01-01,47.0,2024,1,1,monday,True,10:00:00,19:00:00,NaN,...,True,True,True,True,False,False,True,False,Monday,1
295,2024-01-02,16.0,2024,1,2,tuesday,True,10:00:00,19:00:00,NaN,...,True,False,True,True,False,False,True,False,Tuesday,1
296,2024-01-03,20.0,2024,1,3,wednesday,True,10:00:00,19:00:00,NaN,...,True,True,True,True,True,True,True,False,Wednesday,1
297,2024-01-04,62.0,2024,1,4,thursday,True,10:00:00,19:00:00,NaN,...,True,True,True,True,True,True,True,False,Thursday,1
298,2024-01-05,42.0,2024,1,5,friday,True,10:00:00,19:00:00,NaN,...,True,True,True,True,True,False,True,False,Friday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,2024-03-27,26.0,2024,3,27,wednesday,True,10:00:00,19:00:00,NaN,...,True,False,True,True,False,False,False,True,Wednesday,13
381,2024-03-28,31.0,2024,3,28,thursday,True,10:00:00,19:00:00,NaN,...,True,False,True,True,False,False,False,True,Thursday,13
382,2024-03-29,50.0,2024,3,29,friday,True,10:00:00,19:00:00,NaN,...,True,False,True,True,False,False,False,True,Friday,13
383,2024-03-30,55.0,2024,3,30,saturday,True,10:00:00,19:00:00,NaN,...,True,False,True,True,False,False,False,True,Saturday,13


In [46]:
anos = [2023, 2024]
meses = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dec']

for ano in anos:
    for mes in meses:
        print(f"{mes}/{ano}")

jan/2023
fev/2023
mar/2023
abr/2023
mai/2023
jun/2023
jul/2023
ago/2023
set/2023
out/2023
nov/2023
dec/2023
jan/2024
fev/2024
mar/2024
abr/2024
mai/2024
jun/2024
jul/2024
ago/2024
set/2024
out/2024
nov/2024
dec/2024
